In [ ]:
# https://mlflow.org/docs/latest/tracing/integrations/openai-agent
# https://openai.github.io/openai-agents-python/tracing/

In [12]:
from dotenv import load_dotenv
from agents import (

    Agent,
    Runner,
    RunConfig,
    trace,
)


load_dotenv()

True

In [13]:
from helpers.model_client import (
    get_openai_client,
    get_github_model_provider,
)

client = get_openai_client()
GITHUB_MODEL_PROVIDER = get_github_model_provider(client, "gpt-4o-mini")


In [14]:
from helpers.tool_functions import(
    get_current_time,
    get_weather,
)

agent = Agent(
    name="Assistant",
    instructions="Answer the user's questions.",
    tools=[get_weather, get_current_time],

)

In [15]:
# https://github.com/openai/openai-agents-python/blob/main/examples/model_providers/custom_example_provider.py


try:
    with trace(workflow_name="msdiagentic_hello_tracking"):
        result = await Runner.run(
            agent,
            "What is the weather in New York? And what is the time?",
            run_config=RunConfig(model_provider=GITHUB_MODEL_PROVIDER),

        )
        print(result.final_output)
except Exception as e:
    print(f"[error] {e}")


[debug] getting weather for New York
[debug] get_current_time called with location: New York
The weather in New York is currently 4°C and cloudy. The current time is 05:11 PM.


In [16]:
from datetime import datetime


timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
tr = trace(
    workflow_name=f"agent_tracing_workflow_{timestamp}",
    group_id="mdsi",
    metadata={
        "key1": "value1",
        "key2": "value2",
    },
)

# print trace workflow name
print(f"Trace workflow name: {tr.name}")
tr.start()

trace_id = tr.trace_id
print(f"Trace ID: {trace_id}")

run_config = RunConfig(
    model_provider=GITHUB_MODEL_PROVIDER,
    trace_id=trace_id,  # Pass the trace ID to the run config
    trace_metadata={"AgentName": "Weather and Time Agent"},
    workflow_name=tr.name,  # Pass the workflow name to the run config
    group_id=tr.group_id,  # Pass the group ID to the run config
)


Trace workflow name: agent_tracing_workflow_20250330_171159
Trace ID: trace_23736c51983c41a7bf8b810900eb069a


In [17]:
try:
    result = await Runner.run(
            agent,
            "What is the weather in New York? And time.",
            run_config=run_config,
        )

    print(result.final_output)
except Exception as e:
    print(f"[error] {e}")

[debug] getting weather for New York
[debug] get_current_time called with location: New York
The weather in New York is currently 13°C with cloudy conditions. The current time is 05:12 PM.


In [18]:
try:
    result = await Runner.run(
            agent,
            "What is the time in Sydney",
            run_config=run_config,
        )

    print(result.final_output)
except Exception as e:
    print(f"[error] {e}")

[debug] get_current_time called with location: Sydney
The current time in Sydney is 05:12 PM.


In [19]:
try:
    result = await Runner.run(
            agent,
            "Tell me a joke!",
            run_config=run_config,
        )

    print(result.final_output)
except Exception as e:
    print(f"[error] {e}")

Why don’t scientists trust atoms?

Because they make up everything!


In [20]:
tr.finish()

In [21]:
tracing_url = f"https://platform.openai.com/traces/trace?trace_id={trace_id}"

print(f"Tracing URL: {tracing_url}")

Tracing URL: https://platform.openai.com/traces/trace?trace_id=trace_23736c51983c41a7bf8b810900eb069a


View Traces [here](https://platform.openai.com/traces?group_id=Dummy+Group)

# Add Viz

requires GraphViz extension to view

In [11]:
from agents.extensions.visualization import draw_graph

try:
    draw_graph(agent, filename="viz/05_agent_tracing.gv")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH
